# Case of study
- Challenge - Police Racial Bias
- Description: You have beed to understand to invetigate Vehile searches by the police, and if there is racial bias.
1. Create a dataset with these 5 variables : problem, vehicleSearch, race, gender, policePrecint
2. Transform string variables into dummy
3. Get names of Dependent and Independent variables
4. Perform CHAID and visualize. Set max depth to 2

# Imports

In [9]:
# !pip install chaid
# !pip install orca
# !pip install graphviz

In [10]:
# !pip install plotly>=4.0.0
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [11]:
# import orca
import plotly
import pandas as pd
import statsmodels.api as sm 
import plotly.graph_objects as go

from CHAID import Tree

# Load data

In [12]:
data= pd.read_csv('./data/MplsStops.csv').drop(columns='Unnamed: 0')
data.head()

,idNum,date,problem,MDC,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,policePrecinct,neighborhood
0,17-000003,2017-01-01 00:00:42,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Unknown,44.966617,-93.246458,1,Cedar Riverside
1,17-000007,2017-01-01 00:03:07,suspicious,MDC,NaN,NO,NO,Unknown,Unknown,Male,44.980450,-93.271340,1,Downtown West
2,17-000073,2017-01-01 00:23:15,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.948350,-93.275380,5,Whittier
3,17-000092,2017-01-01 00:33:48,suspicious,MDC,NaN,NO,NO,Unknown,East African,Male,44.948360,-93.281350,5,Whittier
4,17-000098,2017-01-01 00:37:58,traffic,MDC,NaN,NO,NO,Unknown,White,Female,44.979078,-93.262076,1,Downtown West


# Data Prep

Select the columns, because are categorical columns:

In [13]:
dataset = data.loc[:, ["problem", "vehicleSearch", "race", "gender", "policePrecinct"]]
dataset.head()                       

,problem,vehicleSearch,race,gender,policePrecinct
0,suspicious,NO,Unknown,Unknown,1
1,suspicious,NO,Unknown,Male,1
2,traffic,NO,White,Female,5
3,suspicious,NO,East African,Male,5
4,traffic,NO,White,Female,1


transform into dummies

In [14]:
dataset = pd.get_dummies(dataset, drop_first = True)
dataset.head()

,policePrecinct,problem_traffic,vehicleSearch_YES,race_Black,race_East African,race_Latino,race_Native American,race_Other,race_Unknown,race_White,gender_Male,gender_Unknown
0,1,0,0,0,0,0,0,0,1,0,0,1
1,1,0,0,0,0,0,0,0,1,0,1,0
2,5,1,0,0,0,0,0,0,0,1,0,0
3,5,0,0,0,1,0,0,0,0,0,1,0
4,1,1,0,0,0,0,0,0,0,1,0,0


# The CHAID Model

get name of Dependent Variable. This dependent variable:


In [15]:
y_name = dataset.loc[:, 'vehicleSearch_YES'].name
y_name

'vehicleSearch_YES'

IV names

In [16]:
X_names = dataset.drop(columns = 'vehicleSearch_YES').columns
print(X_names)
print(dataset.shape)

Index(['policePrecinct', 'problem_traffic', 'race_Black', 'race_East African',
       'race_Latino', 'race_Native American', 'race_Other', 'race_Unknown',
       'race_White', 'gender_Male', 'gender_Unknown'],
      dtype='object')
(51920, 12)


run the model, before we label it the columns if ordinal or nominal:

In [19]:
dict(zip(X_names, ['nominal']* 11))

{'policePrecinct': 'nominal',
 'problem_traffic': 'nominal',
 'race_Black': 'nominal',
 'race_East African': 'nominal',
 'race_Latino': 'nominal',
 'race_Native American': 'nominal',
 'race_Other': 'nominal',
 'race_Unknown': 'nominal',
 'race_White': 'nominal',
 'gender_Male': 'nominal',
 'gender_Unknown': 'nominal'}

In [17]:
model = Tree.from_pandas_df(dataset,
                            dict(zip(X_names, ['nominal']* 11)),
                            y_name,
                            max_depth = 2)

# Visualization

In [18]:
# model.render(path = None,
#              view = None)